# このファイルについて
- author: 松永

## Input
- 関越
    - 全トラカン
        - 2021/04/01-2022/09/30: `関越道TK5M（202104-202209）全トラカン_utf8.CSV`
        - 2022/10/01-2023/07/31: `20221001-20230731_関越道TK5M（全トラカン_関東支社のみ）.csv`
        - 2023/08/01-2023/08/16: `20230801-20230816_TK5M_関越道【全トラカン・関東】.csv`
        - 2023/08/17-2023/09/30: `20230817-20230930_関越道TK5M（全トラカン_上り大泉JCT〜昭和、下り大泉JCT〜沼田）.csv`
        - 2023/10/01-2024/01/31: `20231001-20240131_関越道（上下）TK5M.csv`
        - 2024/02/01-2024/03/31: `20240201-20240331_（関越道）TK5M.csv`
        - 2024/04/01-2024/05/06: `20240401-20240506_（関越道）TK5M.csv`
        - 2024/05/07-2024/08/31: `20240507-20240831_（関越道）TK5M.csv`
    - 代表トラカン
        ...
- 東北
    - 全トラカン
        - 2021/04/01-2021/04/30: `20210401-20210430_東北道_全トラカンTK5M.CSV`
        - 2021/05/01-2021/05/31: `20210501-20210531_東北道_全トラカンTK5M.CSV`
        - 2021/06/01-2021/06/30: `20210601-20210630_東北道_全トラカンTK5M.CSV`
        - 2021/07/01-2021/07/31: `20210701-20210731_東北道_全トラカンTK5M.CSV`
        - 2021/08/01-2021/08/31: `20210801-20210831_東北道_全トラカンTK5M.CSV`
        - 2021/09/01-2021/09/30: `20210901-20210930_東北道_全トラカンTK5M.CSV`
        - 2021/10/01-2021/12/31: `20211001-20211231_東北道_全トラカンTK5M.CSV`
        - 2022/01/01-2022/03/31: `20220101-20220331_東北道_全トラカンTK5M.CSV`
        - 2022/04/01-2023/07/31: `20220401-20230731_東北道TK5M（全トラカン_関東支社のみ）.csv`
        - 2023/08/01-2023/08/16: `20230801-20230816_TK5M_東北道【全トラカン・関東】.csv`
        - 2023/08/17-2023/09/30: `20230817-20230930_東北道TK5M（全トラカン_上り岩槻〜白河、下り岩槻〜矢板）.csv`
        - 2023/10/01-2024/01/31: `20231001-20240131_東北道TK5M（全トラカン_上り岩槻〜白河、下り岩槻〜矢板）.csv`
        - 2024/02/01-2024/03/31: `20240201-20240331_（東北道）TK5M.csv`
        - 2024/04/01-2024/05/06: `20240401-20240506_（東北道）TK5M.csv`
        - 2024/05/07-2024/08/31: `20240507-20240831_（東北道）TK5M.csv`
    - 代表トラカン
        ...
        
## Output
- 全トラカン: `Input_processed_data/traffic/raw_<道路名>_<期間>_2KP.csv`
- 代表トラカン: `Input_processed_data/traffic/raw_<道路名>_<期間>.csv`

# main

1. 文字コードをUTF-8に変換
    - トラカンデータはShift-JIS (sjis)であることが多いため、その場合にUTF-8に事前に変換しておく
2. ヘッダ行を削る
    - トラカンデータは上2行がヘッダ行になっているが、1行目は不要であるため削る

```bash
iconv -f sjis -t utf-8 <csv file> | tail -n +2 > <new file>
```

# appendix

## 好きなように期間を分割する

期間のとり方
- 2021/04/01 〜 2022/03/31
- 2022/04/01 〜 2023/03/31
- 2023/04/01 〜 2023/09/30
- 2023/10/01 ~ 2024/01/31
- 2024/02/01 ~ 2024/05/06

In [6]:
import pandas as pd

In [7]:
TARGET_ROAD = 'kannetsu'
# TARGET_ROAD = 'touhoku'

PERIOD = '20240201-20240506'

RAW_CSV = f'./traffic/raw_{TARGET_ROAD}_{PERIOD}_2KP.csv'

In [8]:
df_raw = pd.read_csv(RAW_CSV, dtype={'年': str, '月': str, '日': str, '時刻': str})

FileNotFoundError: [Errno 2] No such file or directory: './traffic/raw_kannetsu_20240201-20240506_2KP.csv'

In [9]:
import datetime as dt

def add_datetime_columns(df):
    # datetime形式の文字列からdatetime型のカラムを作成
    datetime_str_series = (df.年 + '/' + df.月 + '/' + df.日 + ' ' + df.時刻)
    
    res = df.assign(
        datetime=datetime_str_series.apply(lambda t: dt.datetime.strptime(t, '%Y/%m/%d %H:%M')))
    return res

In [10]:
df_raw_dt = add_datetime_columns(df_raw)
df_raw_dt.head()

,支社,区分,道路,年,月,日,曜日,平休1,平休2（休日に土曜日含む）,時刻,...,OCC(追),速度(追),エラー数.2,全車(路),大車(路),車線率(路),OCC(路),速度(路),エラー数.3,datetime
0,関東支社,速度ループ,関越自動車道,2024,2,1,木,平,平,0:0,...,3.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-01 00:00:00
1,関東支社,速度ループ,関越自動車道,2024,2,1,木,平,平,0:5,...,2.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-01 00:05:00
2,関東支社,速度ループ,関越自動車道,2024,2,1,木,平,平,0:10,...,2.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-01 00:10:00
3,関東支社,速度ループ,関越自動車道,2024,2,1,木,平,平,0:15,...,2.0,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-01 00:15:00
4,関東支社,速度ループ,関越自動車道,2024,2,1,木,平,平,0:20,...,1.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-02-01 00:20:00


In [11]:
df_raw_sorted = df_raw_dt.sort_values(['方向', 'KP', '区間名称', 'datetime']).reset_index(drop=True)

In [12]:
df_raw_sorted.head().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称
0,2024-02-01 00:00:00,2024,2,1,0:0,上り,2.26,大泉ＪＣＴ 〜 所沢
1,2024-02-01 00:05:00,2024,2,1,0:5,上り,2.26,大泉ＪＣＴ 〜 所沢
2,2024-02-01 00:10:00,2024,2,1,0:10,上り,2.26,大泉ＪＣＴ 〜 所沢
3,2024-02-01 00:15:00,2024,2,1,0:15,上り,2.26,大泉ＪＣＴ 〜 所沢
4,2024-02-01 00:20:00,2024,2,1,0:20,上り,2.26,大泉ＪＣＴ 〜 所沢


In [14]:
df_raw_sorted.loc[288:].head().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称
288,2024-02-02 00:00:00,2024,2,2,0:0,上り,2.26,大泉ＪＣＴ 〜 所沢
289,2024-02-02 00:05:00,2024,2,2,0:5,上り,2.26,大泉ＪＣＴ 〜 所沢
290,2024-02-02 00:10:00,2024,2,2,0:10,上り,2.26,大泉ＪＣＴ 〜 所沢
291,2024-02-02 00:15:00,2024,2,2,0:15,上り,2.26,大泉ＪＣＴ 〜 所沢
292,2024-02-02 00:20:00,2024,2,2,0:20,上り,2.26,大泉ＪＣＴ 〜 所沢


In [15]:
df_raw_sorted.tail().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称
3815419,2024-05-06 23:35:00,2024,5,6,23:35,下り,125.37,昭和 〜 沼田
3815420,2024-05-06 23:40:00,2024,5,6,23:40,下り,125.37,昭和 〜 沼田
3815421,2024-05-06 23:45:00,2024,5,6,23:45,下り,125.37,昭和 〜 沼田
3815422,2024-05-06 23:50:00,2024,5,6,23:50,下り,125.37,昭和 〜 沼田
3815423,2024-05-06 23:55:00,2024,5,6,23:55,下り,125.37,昭和 〜 沼田


In [13]:
border = pd.Timestamp('2023/4/1')

In [14]:
df_raw_old = df_raw_sorted.loc[df_raw_sorted.datetime < border].reset_index(drop=True)
df_raw_old.head().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称


In [15]:
df_raw_old.tail().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称


In [16]:
df_raw_new = df_raw_sorted.loc[df_raw_sorted.datetime >= border].reset_index(drop=True)
df_raw_new.head().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称
0,2023-10-01 00:00:00,2023,10,1,0:0,上り,2.26,大泉ＪＣＴ 〜 所沢
1,2023-10-01 00:05:00,2023,10,1,0:5,上り,2.26,大泉ＪＣＴ 〜 所沢
2,2023-10-01 00:10:00,2023,10,1,0:10,上り,2.26,大泉ＪＣＴ 〜 所沢
3,2023-10-01 00:15:00,2023,10,1,0:15,上り,2.26,大泉ＪＣＴ 〜 所沢
4,2023-10-01 00:20:00,2023,10,1,0:20,上り,2.26,大泉ＪＣＴ 〜 所沢


In [17]:
df_raw_new.tail().loc[:, ['datetime', '年', '月', '日', '時刻', '方向', 'KP', '区間名称']]

,datetime,年,月,日,時刻,方向,KP,区間名称
4888507,2024-01-31 23:35:00,2024,1,31,23:35,下り,125.37,昭和 〜 沼田
4888508,2024-01-31 23:40:00,2024,1,31,23:40,下り,125.37,昭和 〜 沼田
4888509,2024-01-31 23:45:00,2024,1,31,23:45,下り,125.37,昭和 〜 沼田
4888510,2024-01-31 23:50:00,2024,1,31,23:50,下り,125.37,昭和 〜 沼田
4888511,2024-01-31 23:55:00,2024,1,31,23:55,下り,125.37,昭和 〜 沼田


In [18]:
assert len(df_raw_old) + len(df_raw_new) == len(df_raw)

In [19]:
RAW_CSV

'./traffic/raw_kannetsu_20231001-20240131_2KP.csv'

In [42]:
period_old = '20220401-20230331'
period_new = '20230401-20230731'

file_old = f'./traffic/raw_{TARGET_ROAD}_{period_old}_2KP.csv'
file_new = f'./traffic/raw_{TARGET_ROAD}_{period_new}_2KP.csv'

In [20]:
# %time df_raw_old.drop('datetime', axis=1).to_csv(file_old, index=False)

In [21]:
# %time df_raw_new.drop('datetime', axis=1).to_csv(file_new, index=False)